In [95]:
# Importing Libraries
import pandas as pd
import numpy as np


In [96]:
# Loading Datasets
df = pd.read_csv("/content/drive/My Drive/productivity/datasets/football_players.csv")
df.head()

,name,club,age,position,position_cat,market_value,page_views,fpl_value,fpl_sel,fpl_points,region,nationality,new_foreign,age_cat,club_id,big_club,new_signing
0,Alexis Sanchez,Arsenal,28,LW,1,65.0,4329,12.0,17.10%,264,3.0,Chile,0,4,1,1,0
1,Mesut Ozil,Arsenal,28,AM,1,50.0,4395,9.5,5.60%,167,2.0,Germany,0,4,1,1,0
2,Petr Cech,Arsenal,35,GK,4,7.0,1529,5.5,5.90%,134,2.0,Czech Republic,0,6,1,1,0
3,Theo Walcott,Arsenal,28,RW,1,20.0,2393,7.5,1.50%,122,1.0,England,0,4,1,1,0
4,Laurent Koscielny,Arsenal,31,CB,3,22.0,912,6.0,0.70%,121,2.0,France,0,4,1,1,0


<h2> Investigating the dataset

In [97]:
df.shape

(461, 17)

In [98]:
df.describe(include='all')

,name,club,age,position,position_cat,market_value,page_views,fpl_value,fpl_sel,fpl_points,region,nationality,new_foreign,age_cat,club_id,big_club,new_signing
count,461,461,461.000000,461,461.000000,461.000000,461.000000,461.000000,461,461.000000,460.000000,461,461.000000,461.000000,461.000000,461.000000,461.000000
unique,461,20,NaN,13,NaN,NaN,NaN,NaN,113,NaN,NaN,61,NaN,NaN,NaN,NaN,NaN
top,Craig Cathcart,Huddersfield,NaN,CB,NaN,NaN,NaN,NaN,0.10%,NaN,NaN,England,NaN,NaN,NaN,NaN,NaN
freq,1,28,NaN,85,NaN,NaN,NaN,NaN,64,NaN,NaN,156,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,26.804772,NaN,2.180043,11.012039,763.776573,5.447939,NaN,57.314534,1.993478,NaN,0.034707,3.206074,10.334056,0.303688,0.145336
std,NaN,NaN,3.961892,NaN,1.000061,12.257403,931.805757,1.346695,NaN,53.113811,0.957689,NaN,0.183236,1.279795,5.726475,0.460349,0.352822
min,NaN,NaN,17.000000,NaN,1.000000,0.050000,3.000000,4.000000,NaN,0.000000,1.000000,NaN,0.000000,1.000000,1.000000,0.000000,0.000000
25%,NaN,NaN,24.000000,NaN,1.000000,3.000000,220.000000,4.500000,NaN,5.000000,1.000000,NaN,0.000000,2.000000,6.000000,0.000000,0.000000
50%,NaN,NaN,27.000000,NaN,2.000000,7.000000,460.000000,5.000000,NaN,51.000000,2.000000,NaN,0.000000,3.000000,10.000000,0.000000,0.000000
75%,NaN,NaN,30.000000,NaN,3.000000,15.000000,896.000000,5.500000,NaN,94.000000,2.000000,NaN,0.000000,4.000000,15.000000,1.000000,0.000000


In [99]:
cat = [x for x in list(df.select_dtypes(exclude=['int', 'float']).columns)]
print("Categorical Features:")
for i in range(len(cat)):
  print(str(i+1)+". "+cat[i])

Categorical Features:
1. name
2. club
3. position
4. fpl_sel
5. nationality


In [100]:
num = [x for x in list(df.select_dtypes(include=['int', 'float']).columns)]
print("Numerical Features:")
for i in range(len(num)):
  print(str(i+1)+". "+num[i])

Numerical Features:
1. age
2. position_cat
3. market_value
4. page_views
5. fpl_value
6. fpl_points
7. region
8. new_foreign
9. age_cat
10. club_id
11. big_club
12. new_signing


<h2> 1. Cleaning and Transforming the data

In [101]:
# Filling the missing values
df.isnull().sum()

name            0
club            0
age             0
position        0
position_cat    0
market_value    0
page_views      0
fpl_value       0
fpl_sel         0
fpl_points      0
region          1
nationality     0
new_foreign     0
age_cat         0
club_id         0
big_club        0
new_signing     0
dtype: int64

In [102]:
df['region'] = df['region'].fillna(2.0)

In [104]:
# Converting % string values of fpl_sel to numerical values

for i in range(len(df['fpl_sel'])):
  df['fpl_sel'][i] = float(df['fpl_sel'][i][:-1])

df['fpl_sel'] = pd.to_numeric(df['fpl_sel'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [105]:
df['fpl_sel']

0      17.1
1       5.6
2       5.9
3       1.5
4       0.7
       ... 
456     0.4
457     0.2
458     0.3
459     5.9
460     0.1
Name: fpl_sel, Length: 461, dtype: float64